<a href="https://colab.research.google.com/github/kkkkkkkm/Torch/blob/main/CIFAR-100/CIFAR-100_CNN_without_setting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 순서
# 일반적으로 수행, finetuning, freeze 후 분류기 학습.
# dropout 추가 
# 정규화
# augmentaion

In [28]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data.dataloader import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from torchvision.utils import make_grid
import torch.nn.functional as F
from torch.optim import lr_scheduler
import torch.optim as optim
import time
import os
import copy

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)

if device == 'cuda':
  torch.cuda.manual_seed_all(777)

print("기기 사용 {}".format(device))

기기 사용 cuda


In [3]:
transform = transforms.Compose([
                                transforms.ToTensor()
])

In [ ]:
trainset = torchvision.datasets.CIFAR100(root = './cifar-100',
                                         train = True,
                                         download = True,
                                         transform = transform)


In [5]:
print(trainset.data.shape)

(50000, 32, 32, 3)


In [ ]:
train_data_mean = trainset.data.mean(axis = (0,1,2)) # 각 축에대한 평균 
train_data_std = trainset.data.std(axis = (0,1,2)) # 각 축에 대한 분산 ()

print(f"train_mean : {train_data_mean}")
print(f"train_std : {train_data_std}")

train_data_mean = (train_data_mean/ 255.)
train_data_std = (train_data_std/ 255.)

print(train_data_mean)
print(train_data_std)


In [ ]:
transform_train = transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize(train_data_mean, train_data_std)
])

transform_test = transforms.Compose([
                                     transforms.ToTensor(),
                                     transforms.Normalize(train_data_mean , train_data_std)
])


train_set = torchvision.datasets.CIFAR100(
    root = './cifar-100',
    train = True,
    download = True,
    transform = transform_train
)

test_set = torchvision.datasets.CIFAR100(
    root = './cifar-100',
    train = False,
    download = True,
    transform = transform_test 
)

train_loader = torch.utils.data.DataLoader(train_set, 
                                           batch_size = 256,
                                           shuffle = True,
                                           num_workers = 0)

test_loader = torch.utils.data.DataLoader(test_set,
                                          batch_size= 256,
                                          num_workers = 0)



In [8]:
import torchvision.models as models

In [9]:
test_model = models.resnet50(pretrained= True)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
test_model

In [11]:
test_model.fc = nn.Linear(2048, 1024)

In [12]:
class mymodel(nn.Module):
  def __init__(self):
    super(mymodel, self).__init__()
    self.linear1 = nn.Linear(1024, 512)
    self.linear2 = nn.Linear(512, 256)
    self.linear3 = nn.Linear(256, 100)
    self.relu = nn.ReLU(inplace= True)
    
    self.layer1 = torch.nn.Sequential(
        self.linear1,
        self.relu,
        self.linear2,
        self.relu,
        self.linear3
    )
    

  def forward(self, x):
    out = self.layer1(x)

    return out
    
  


In [13]:
my_model = mymodel()

In [14]:
my_model

mymodel(
  (linear1): Linear(in_features=1024, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=256, bias=True)
  (linear3): Linear(in_features=256, out_features=100, bias=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=256, out_features=100, bias=True)
  )
)

In [15]:
total_model = nn.Sequential(
    test_model,
    my_model
)

In [20]:
total_model = total_model.to(device)

In [145]:
test_image = torch.rand(256,3,32,32)

print(total_model(test_image).shape)

torch.Size([256, 100])


In [29]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(total_model.parameters(), lr = 0.001)
lr_scheduler = lr_scheduler.StepLR(optimizer, step_size = 4, gamma= 0.3)

In [32]:
for epoch in range(10):
  avg_cost = 0
  total_batch = len(train_loader)
  for x , y in train_loader:
    optimizer.zero_grad()
    x = x.to(device)
    y = y.to(device)
    
    h = total_model(x)
    
    loss = criterion(h , y)
    loss.backward()
    optimizer.step()

    avg_cost += loss / total_batch
  print("epoch: {} , cost = {:.9f}".format(epoch, avg_cost))

epoch: 0 , cost = 1.122426510
epoch: 1 , cost = 0.896077633
epoch: 2 , cost = 0.748636484
epoch: 3 , cost = 0.694267333
epoch: 4 , cost = 0.560615420
epoch: 5 , cost = 0.617601514
epoch: 6 , cost = 0.469593674
epoch: 7 , cost = 0.338274568
epoch: 8 , cost = 0.391688287
epoch: 9 , cost = 0.578212559


In [33]:
with torch.no_grad():
  total_model.eval()
  acc =0 
  avg_test_loss = 0
  correct = 0
  for data, target in test_loader:
    data = data.to(device)
    target = target.to(device)
    
    h = total_model(x)
    
    test_loss = criterion(h, target) 
    
    prediction = total_model(data)
    correct_predict = torch.argmax(prediction, 1)

    avg_test_loss += test_loss / len(test_loader)

    acc += correct_predict.eq(target.view_as(correct_predict)).sum().item()

  
  print("acc: {}/{} ({:.2f}%)".format(acc, len(test_loader.dataset), (acc/len(test_loader.dataset)) * 100. ))


acc: 5177/10000 (51.77%)
